# **GROUP PROJECT**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from collections import defaultdict
import geopandas as gpd
import random
from deap import base, creator, tools
import pyproj
from shapely.geometry import Point
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import ortools as ort
import datetime as dt
# pysal submodule imports
from pysal.lib import cg, examples
from pysal.explore import spaghetti as spgh

import geopandas as gpd
from shapely.geometry import Point
from ortools.linear_solver import pywraplp
import copy, sys, warnings
from collections import OrderedDict

import matplotlib as mpl
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

try:
    from IPython.display import set_matplotlib_formats
    set_matplotlib_formats('retina')
except ImportError:
    pass

# **1. Problem Variables**

In [ ]:
# truck variables
truckRange = 10
truckCapacity = 500
nCities = 40
M = 30

# **2. SET UP CITIES**

In [ ]:
def buildCityDF(nCities,gridX=100,gridY=100):
    xs = np.random.randint(0,gridX,nCities)
    ys = np.random.randint(0,gridY,nCities)
    pts = [Point(xs[i],ys[i]) for i in range(0,nCities)]
    demands = np.random.randint(0,101,nCities)
    cityIndices = range(0,nCities)
    cities = gpd.GeoDataFrame({"x":xs,"y":ys,"demand":demands,"geometry":pts,"City Index":cityIndices})
    return cities

In [ ]:
cities = buildCityDF(nCities)

# **3. DIVIDE UP CITIES INTO SUB-TOURS TO BE SOLVED USING TSP**

In [ ]:
def buildDistanceMatrix_Euclidean(cities):
    distances = np.zeros((len(cities),len(cities)))
    for i,row in cities.iterrows():
        for j,row2 in cities.iterrows():
            distances[j,i] = row.geometry.distance(row2.geometry)
    return distances

In [ ]:
def solveOriginal(cities,M = 100):
    solver_instance = pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING
    distances = buildDistanceMatrix_Euclidean(cities)
    cij = distances
    n_customers = cij.shape[0]
    range_customers = range(n_customers)
    n_facilityCandidates = cij.shape[1]
    range_facilityCandidates = range(n_facilityCandidates)
    demands = np.array(cities["demand"])
    demands_sum = demands.sum()

    model = pywraplp.Solver("Capacitated Fixed Charge", solver_instance)
    
    xij = {(i,j): model.IntVar(0,1, 'x[%i,%i]' % (i,j)) for i in range_customers for j in range_facilityCandidates}
    yj = {j: model.IntVar(0,1, 'y[%i]' % (j)) for j in range_facilityCandidates}
    
    obj = [(cij[0,j]+cij[i,j]+cij[i,0]) * xij[i,j] + 2*cij[0,j]*yj[j] for i in range_customers for j in range_facilityCandidates]
    
    for i in range_customers:
        model.Add(model.Sum([xij[i,j] for j in range_facilityCandidates]) == 1)
        
    for j in range_facilityCandidates:
        model.Add(model.Sum([demands[i] * xij[i,j] for i in range_facilityCandidates]) <= truckCapacity)
            
    for i in range_customers:
        for j in range_facilityCandidates:
            model.Add(xij[i,j] <= yj[j])
            
    for j in range_facilityCandidates:
        #model.Add(yj[j] <= M)
        model.Add(model.Sum([yj[j] for j in range_facilityCandidates]) <= M)
        

    model.Minimize(model.Sum(obj))
    status = model.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Optimal Solution Found')

    resultGDF = cities.copy(deep=True)
    resultGDF["Status"] = [yj[j].solution_value() for j in range_facilityCandidates]
    
    return model,yj,xij,resultGDF,status

In [ ]:
m,yj,xij,resultGDF,status = solveOriginal(cities,M=M)

In [ ]:
assignments = np.zeros((len(cities),len(cities)))
for k,v in xij.items():
    i = k[0]
    j = k[1]
    assignments[i,j] = v.solution_value()
cluster = []
for cityNodeID in range(len(cities)):
    for rootNodeID in range(len(cities)):
        lookup = assignments[cityNodeID,rootNodeID]
        if lookup == 1:
            cluster.append(rootNodeID)
resultGDF["assignment"] = cluster
uniqueClusters = list(set(cluster))

fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
fig.tight_layout()
colors = ['#'+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(uniqueClusters))]
for i in range(0,len(uniqueClusters)):
    dfSubset = resultGDF[resultGDF["assignment"]==uniqueClusters[i]]
    dfSubset.plot(ax=ax1,color=colors[i],markersize=dfSubset["demand"],label=f"({cities.iloc[uniqueClusters[i]].x},{cities.iloc[uniqueClusters[i]].y})")
ax1.set_title("Tours organized by cluster\nNode sizes proportional to demand")
ax1.legend()
# Plot assignments
plt.imshow(assignments)
ax2.set_title("Assignment of citites to subtours")
ax2.set_xlabel("Subtour Root (Flag/Concentrator) Node")
ax2.set_ylabel("City")

# **4. SOLVE TSP SUB-TOURS**

In [74]:
def buildEdgeDf_Geo(df):
    edgeIDs = []
    sources = []
    destinations = []
    weights = []
    edgeID = 0
    for i,row in df.iterrows():
        sourceID = row["City Index"]
        sourceGeometry = row["geometry"]
        for j,row in df.iterrows():
            if j > i:
                destID = row["City Index"]
                destGeometry = row["geometry"]
                distance = sourceGeometry.distance(destGeometry)
                edgeIDs.append(edgeID)
                sources.append(sourceID)
                destinations.append(destID)
                weights.append(distance)
                edgeID += 1
    df_links = pd.DataFrame({"Edge":edgeIDs,"Source":sources,"Destination":destinations,"weight":weights})
    return df_links

In [124]:
def buildMinimumSpanningTree(df):
    """
    Utilizes Kruskal's algorithm
    """
    # 0. create dataframe of edges with weight, source, and destination
    if type(df) == gpd.geodataframe.GeoDataFrame:
        df_links = buildEdgeDf_Geo(df).sort_values("weight")
    else:
        df_links = buildEdgeDf(df).sort_values("weight")
    
    # 1. load all nodes into a graph
    graph = nx.Graph()
    graph.add_nodes_from(df.index)
    # 2. Implement Kruskal's algorithm by adding each edge in order by weight so long as it does not create a cycle
    for i,row in df_links.iterrows():
        verticesReached = []
        source = row["Source"]
        destination = row["Destination"]
        weight = row["weight"]
        graph.add_edge(source,destination)
        cyclesInGraph = nx.cycle_basis(graph)
        if len(cyclesInGraph) > 0: # check for cycle and remove edge if one is created
            graph.remove_edge(source,destination)
        if len(graph.edges) == len(graph.nodes) - 1:
            break
    # check results
    fullgraph = nx.from_pandas_edgelist(df_links,source="Source",target="Destination",edge_attr="weight")
    mst = nx.minimum_spanning_tree(fullgraph)
    isSameGraph = nx.is_isomorphic(graph,mst)
    print(f"The MST algorithm matches that of NetworkX: {isSameGraph}")
    return df_links,graph

In [125]:
# duplicate every edge on the MST
def duplicateEdges(df_links):
    edgeID = max(df_links["Edge"]) + 1
    for i,row in df_links.iterrows():
        destination = row["Destination"] # flip these because directed!
        source = row["Source"] # flip these because directed!
        weight = row["weight"]
        df_links = df_links.append({"Edge":edgeID,"Source":destination,"Destination":source,"weight":weight},ignore_index=True)
        edgeID += 1
    df_links.index = df_links["Edge"]
    return df_links

In [126]:
def findNearestNodeNotAlreadyInSet(currentNodeID,df_links_geo_bidirectional,citiesNotInPath):
    df_edgesStartingAtCity = df_links_geo_bidirectional[df_links_geo_bidirectional["Source"]==currentNodeID].sort_values("weight")
    for i,row in df_edgesStartingAtCity.iterrows():
        destinationID = row["Destination"]
        if destinationID in citiesNotInPath:
            #addCityToPath(destinationID)
            return destinationID

In [127]:
def addCityToPath(citiesInPath,citiesNotInPath,cityID):
    citiesInPath.append(cityID)
    citiesNotInPath.remove(cityID)
    return citiesInPath,citiesNotInPath

In [128]:
dfTourSubset = resultGDF[resultGDF["assignment"]==0]

In [129]:
df_links_geo,graph_geo = buildMinimumSpanningTree(dfTourSubset)

The MST algorithm matches that of NetworkX: True


In [130]:
df_links_geo_bidirectional = duplicateEdges(df_links_geo)

In [ ]:
startingPopulation = []
for startNode in range(0,len(cities)):
    citiesInPath = []
    citiesNotInPath = [i for i in range(0,len(cities))]
    currentNode = startNode
    while len(citiesNotInPath) > 0:
        addCityToPath(currentNode)
        nextNode = findNearestNodeNotAlreadyInSet(currentNode)
        currentNode = nextNode
    citiesInPath = [int(cityID) for cityID in citiesInPath]
    startingPopulation.append(citiesInPath)

In [141]:
def generateStartingPopulation(dfTourSubset):
    startingPopulation = []
    for startNode in dfTourSubset["City Index"]:
        citiesInPath = []
        citiesNotInPath = [cityIndex for cityIndex in dfTourSubset["City Index"]]
        currentNode = startNode
        while len(citiesNotInPath) > 0:
            citiesInPath,citiesNotInPath = addCityToPath(citiesInPath,citiesNotInPath,currentNode)
            nextNode = findNearestNodeNotAlreadyInSet(currentNode,df_links_geo_bidirectional,citiesNotInPath)
            currentNode = nextNode
        citiesInPath = [int(cityID) for cityID in citiesInPath]
        startingPopulation.append(citiesInPath)
    return startingPopulation

In [142]:
generateStartingPopulation(dfTourSubset)

[[0, 35, 19, 27, 39, 23, 16, 37, 36, 26, 38, 24, 33, 25, 29, 15, 4],
 [4, 15, 29, 25, 33, 24, 23, 39, 16, 37, 36, 26, 38, 0, 35, 19, 27],
 [15, 29, 25, 33, 24, 23, 39, 16, 37, 36, 26, 38, 0, 35, 19, 27, 4],
 [16, 37, 36, 26, 38, 24, 33, 0, 35, 19, 27, 39, 23, 25, 29, 15, 4],
 [19, 0, 35, 33, 24, 23, 39, 16, 37, 36, 26, 38, 29, 25, 15, 4, 27],
 [23, 39, 16, 37, 36, 26, 38, 24, 33, 0, 35, 19, 27, 25, 29, 15, 4],
 [24, 33, 0, 35, 19, 27, 39, 23, 16, 37, 36, 26, 38, 29, 25, 15, 4],
 [25, 29, 15, 4, 35, 0, 19, 27, 39, 23, 16, 37, 36, 26, 38, 24, 33],
 [26, 38, 36, 37, 16, 23, 39, 27, 0, 35, 19, 33, 24, 25, 29, 15, 4],
 [27, 39, 23, 16, 37, 36, 26, 38, 24, 33, 0, 35, 19, 4, 15, 29, 25],
 [29, 25, 15, 4, 35, 0, 19, 27, 39, 23, 16, 37, 36, 26, 38, 24, 33],
 [33, 24, 23, 39, 16, 37, 36, 26, 38, 29, 25, 15, 4, 35, 0, 19, 27],
 [35, 0, 19, 27, 39, 23, 16, 37, 36, 26, 38, 24, 33, 25, 29, 15, 4],
 [36, 26, 38, 37, 16, 23, 39, 27, 0, 35, 19, 33, 24, 25, 29, 15, 4],
 [37, 16, 36, 26, 38, 24, 33, 0, 3

# **5. SOLVE FOR CHARGING STATIONS**

# **A) CLEAN UP, CLEAN UP, EVERYBODY EVERYWHERE**

By Tomorrow
- TSP code  with GA
- Test segementation code for size, loop through and see how long it takes to run

# **CHECK HOW LARGE WE CAN DO**

In [ ]:
nCities = range(10,110,10)
gridX = 100
gridY = 100

In [ ]:
times = []
for nCity in nCities:
    startTime = dt.datetime.now()
    cities = buildCityDF(nCity,gridX=100,gridY=100)
    m,yj,xij,resultGDF,status = solveOriginal(cities)
    endTime = dt.datetime.now()
    totalTime = (endTime-startTime).total_seconds()
    times.append(totalTime)
    print(f"Completed {nCity} cities in {totalTime} seconds.  Optimal solution: {status == pywraplp.Solver.OPTIMAL}")